# Image generation with Stable Diffusion

In [4]:
import os
import io
import requests
from PIL import Image
from IPython.display import display
from IPython.display import Image as IPythonImage
import gradio as gr

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv(filename="secrets.env", raise_error_if_not_found=True))
ROOT_DIR = os.environ["ROOT_DIR"]
HF_API_TOKEN = os.environ["HF_API_TOKEN"]
HF_MODEL = "runwayml/stable-diffusion-v1-5"
API_URL = f"https://api-inference.huggingface.co/models/{HF_MODEL}"
HEADERS = {"Authorization": f"Bearer {HF_API_TOKEN}"}

## Test the API

In [ ]:
def query(payload):
	response = requests.post(API_URL, headers=HEADERS, json=payload)
	image_bytes = response.content
	return image_bytes

image_bytes = query({
	"inputs": "a cat on the back of a dinausor",
})

image = Image.open(io.BytesIO(image_bytes))
path = ROOT_DIR + "/files/gen_img.png"
image.save(path, format="png")
IPythonImage(path)
display(IPythonImage(image_bytes))

## Define and test the completion function

## Create a simple app usiong Gradio Interface

In [ ]:
# Define the API call
def generate(prompt:str):
    response = requests.post(url=API_URL, headers=HEADERS, json=prompt)
    return Image.open(io.BytesIO(response.content))

gr.close_all()
app = gr.Interface(
    fn=generate,
    inputs=gr.Textbox(label="Enter your prompt"),
    outputs=gr.Image(label="Generated image", type="pil"),
    title="Image generator",
    description="Generate an image for a given prompt, powered by 'stable-diffusion-v1-5'",
    allow_flagging="never",
    examples=["a friendly lama eating popcorn and watching a movie, in a cartoon style"],
)

app.launch(share=True)

## Creating a more advanced app using Blocks and Accordion

In [5]:
# update the API call to take additional parameters
def API_call(inputs:str, parameters = None, url = API_URL):
	
	# create the payload (the prompt)
	payload = {"inputs": inputs}
	
	# update it with more detailled parameters if available
	if parameters != None:
		payload.update({"parameters": parameters})
	
	# send the request
	response = requests.post(url=url, json=payload, headers=HEADERS)

	# extract the output from the response
	imageBytes = response.content

	# return it
	return imageBytes

# update the generate function to accept these additional parameters
def generate(prompt:str, negative_prompt:str="", steps:int=25, guidance:int=7, width:int=512, height:int=512):
	# format the parameters for the API call
	parameters = {
		"negative_prompt" : negative_prompt,
		"num_inference_steps": steps,
		"guidance_scale": guidance,
		"width": width,
		"height": height,
	}

	# make the API call
	ImageByte = API_call(inputs=prompt, parameters=parameters)
	
	# convert to PIL format
	output = Image.open(io.BytesIO(ImageByte))

	# return the output
	return output

# Create the advanced app
with gr.Blocks(theme = gr.themes.Monochrome()) as app:
	# Title and description
	gr.Markdown("# Image generation app")
	gr.Markdown("Powered by stable-diffusion-v1.5 under the hood")
	
	# First row: prompt and button side by side
	with gr.Row():
		with gr.Column(scale=5):
			prompt = gr.Textbox(label="Enter your prompt", scale=5)
		with gr.Column(scale=1):
			button = gr.Button(value="submit", min_width=30, scale=1)
	
	# Second row: allow for advanced customization
	with gr.Accordion(label="Advanced option", open=False): # should not be visible by default
		
		# first row: negative prompt
		negativePrompt = gr.Textbox(label="Negative Prompt", value="low quality")

		# second row: two columns of advanced options
		with gr.Row():
			with gr.Column():
				gr.Markdown("Parameters for the image generator")
				steps = gr.Slider(
					label="Inference steps",
					minimum=1,
					maximum=100,
					value=25,
					step=1,
					info = "In how many steps should the denoiser generate the image?"
				)
				guidance = gr.Slider(
					label="Generation guidance",
					minimum=1,
					maximum=25,
					value=7,
					step=1,
					info = "How much should the prompt influence the generation?"
				)
			with gr.Column():
				gr.Markdown("Size of the generated image")
				width = gr.Slider(
					label="Width",
					minimum=64,
					maximum=512,
					step=64,
					value=512,
					info="Expected width of the generated image",
				)
				height = gr.Slider(
					label="Height",
					minimum=64,
					maximum=512,
					step=64,
					value=512,
					info="Expected height of the generated image",
				)
	# next row: result (generated image)
	outputs = gr.Image(label="Result", type="pil")
	
	# define the logic of the app
	button.click(
		fn=generate,
		inputs=[prompt, negativePrompt, steps, guidance, width, height],
		# inputs=[prompt],
		outputs=outputs,
	)

# gr.close_all()
app.launch()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


## Testing image generation with DALL-E

In [ ]:
from openai import OpenAI
client = OpenAI()
client.api_key = os.environ["OPENAI_API_KEY"]

prompt = "A futuristic vision of the city of Munich in year 2100"

response = client.images.generate(
  model="dall-e-3",
  prompt=prompt,
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url
IPythonImage(url=image_url)